# 6. Tuning

In [1]:
MODEL = 'EBR'
TUNING_MODEL = 'guanganb'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAnB_H2H

In [11]:
guangan = GuangAnB_H2H.GuangAnB()

GuangAn Initialised


In [12]:
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0

In [13]:
from interpret.glassbox import ExplainableBoostingRegressor as EBR

In [14]:
guangan.read_in_model(EBR, 'Regression')

Successfully read in model <class 'interpret.glassbox._ebm._ebm.ExplainableBoostingRegressor'>, which is a Regression model


In [15]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [16]:
parameter_choices = {
    'max_bins': set(range(4, 4097)),
    'min_samples_leaf': set(range(2, 33)),
    'interactions': set(range(2, 9)),
    'max_leaves': (2, 4),
    'learning_rate': (-6, 1),
    'max_rounds': set(range(25, 201)),

}

In [17]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [18]:
guangan.read_in_transform({'learning_rate': '10^', 'max_leaves': '10^'})

Updated transform dictionary: {'max_bins': False, 'min_samples_leaf': False, 'interactions': False, 'max_leaves': '10^', 'learning_rate': '10^', 'max_rounds': False}


In [19]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

Successfully recorded non_tuneable_hyperparameter choices


In [20]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [21]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 8059 rows
True


In [22]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Has trained 8059 combinations so far
Already Trained and Tested combination {'max_bins': 2306, 'min_samples_leaf': 12, 'interactions': 6, 'max_leaves': 2.375, 'learning_rate': -0.3125, 'max_rounds': 168, 'features': 5}, which had val score of 0.5839
        Current best combo: (2306, 12, 6, 2.125, -0.3125, 168, 3) with val score 0.6153. 
        Has trained 8059 combinations so far
Already Trained and Tested combination {'max_bins': 2306, 'min_samples_leaf': 12, 'interactions': 6, 'max_leaves': 2.375, 'learning_rate': -0.3125, 'max_rounds': 179, 'features': 3}, which had val score of 0.6122
        Current best combo: (2306, 12, 6, 2.125, -0.3125, 168, 3) with val score 0.6153. 
        Has trained 8059 combinations so far
Already Trained and Tested combination {'max_bins': 2306, 'min_samples_leaf': 12, 'interactions': 6, 'max_leaves': 2.375, 'learning_rate': -0.3125, 'max_rounds': 179, 'features': 5}, which had val score of 0.

In [27]:
guangan.view_best_combo_and_score()

Max Score: 
 0.6153039832285115
Max Test Score: 
 0.615304
Max Combo Index: 
 (2306, 12, 6, 2.125, -0.3125, 168, 3)
Max Combo Hyperparamer Combination: 
 {'max_bins': 2306, 'min_samples_leaf': 12, 'interactions': 6, 'max_leaves': 2.125, 'learning_rate': -0.3125, 'max_rounds': 168, 'features': 3}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts')
# Combos Checked: 8059
